# Interactive 3D Camera Exploration for RenderMe360 s3_all

This notebook provides interactive 3D visualization of camera positions.
You can rotate, zoom, and explore the camera arrangement.

In [ ]:
# Import required libraries
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML

print("Libraries loaded successfully!")

## 1. Load Camera Calibration Data

In [ ]:
def load_camera_data(subject_id="0026"):
    """Load camera calibration data and compute positions."""
    
    # Load calibration file
    base_dir = Path(f'/ssd4/zhuoyuan/renderme360_temp/download_all/subjects/{subject_id}')
    calib_file = base_dir / 's3_all' / 'calibration' / 'all_cameras.npy'
    
    if not calib_file.exists():
        print(f"Calibration file not found: {calib_file}")
        return None
    
    calibs = np.load(calib_file, allow_pickle=True).item()
    
    # Extract camera positions and compute metrics
    camera_data = []
    
    for cam_str, calib in calibs.items():
        cam_id = int(cam_str)
        RT = calib['RT']
        R = RT[:3, :3]
        t = RT[:3, 3]
        
        # Camera position (using correct method)
        cam_pos = -t
        
        # Calculate spherical coordinates
        x, y, z = cam_pos
        r_xy = np.sqrt(x**2 + y**2)  # radial distance in XY plane
        r_total = np.sqrt(x**2 + y**2 + z**2)  # total distance
        theta = np.degrees(np.arctan2(y, x))  # azimuth angle
        phi = np.degrees(np.arctan2(z, r_xy))  # elevation angle
        
        # Camera viewing direction (optical axis)
        optical_axis = R[2, :]  # Z-axis of camera in world coordinates
        
        camera_data.append({
            'id': cam_id,
            'x': x,
            'y': y,
            'z': z,
            'r_xy': r_xy,
            'r_total': r_total,
            'azimuth': theta,
            'elevation': phi,
            'optical_x': optical_axis[0],
            'optical_y': optical_axis[1],
            'optical_z': optical_axis[2]
        })
    
    return pd.DataFrame(camera_data).sort_values('id')

# Load camera data
camera_df = load_camera_data()
print(f"Loaded {len(camera_df)} cameras")
print("\nCamera data sample:")
display(camera_df.head())

## 2. Interactive 3D Visualization with Plotly

In [ ]:
# Create interactive 3D scatter plot
fig = go.Figure()

# Add cameras as scatter points
fig.add_trace(go.Scatter3d(
    x=camera_df['x'],
    y=camera_df['y'],
    z=camera_df['z'],
    mode='markers+text',
    marker=dict(
        size=8,
        color=camera_df['z'],  # Color by height
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Height (m)"),
        line=dict(width=1, color='black')
    ),
    text=camera_df['id'].astype(str),
    textposition="top center",
    textfont=dict(size=10),
    hovertemplate='<b>Camera %{text}</b><br>' +
                  'X: %{x:.2f}m<br>' +
                  'Y: %{y:.2f}m<br>' +
                  'Z: %{z:.2f}m<br>' +
                  '<extra></extra>',
    name='Cameras'
))

# Add subject at origin
fig.add_trace(go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    marker=dict(
        size=15,
        color='red',
        symbol='diamond'
    ),
    name='Subject',
    hovertemplate='<b>Subject Position</b><br>Origin (0,0,0)<extra></extra>'
))

# Add cylindrical wireframe to show the arrangement
theta_wire = np.linspace(0, 2*np.pi, 30)
z_wire = np.linspace(camera_df['z'].min(), camera_df['z'].max(), 10)
avg_radius = camera_df['r_xy'].mean()

# Add circular rings at different heights
for z_level in np.linspace(camera_df['z'].min(), camera_df['z'].max(), 5):
    x_circle = avg_radius * np.cos(theta_wire)
    y_circle = avg_radius * np.sin(theta_wire)
    z_circle = np.full_like(theta_wire, z_level)
    
    fig.add_trace(go.Scatter3d(
        x=x_circle,
        y=y_circle,
        z=z_circle,
        mode='lines',
        line=dict(color='gray', width=1),
        opacity=0.3,
        showlegend=False,
        hoverinfo='skip'
    ))

# Add vertical lines
for angle in np.linspace(0, 2*np.pi, 8, endpoint=False):
    x_line = [avg_radius * np.cos(angle)] * 2
    y_line = [avg_radius * np.sin(angle)] * 2
    z_line = [camera_df['z'].min(), camera_df['z'].max()]
    
    fig.add_trace(go.Scatter3d(
        x=x_line,
        y=y_line,
        z=z_line,
        mode='lines',
        line=dict(color='gray', width=1),
        opacity=0.3,
        showlegend=False,
        hoverinfo='skip'
    ))

# Update layout
fig.update_layout(
    title=dict(
        text='Interactive 3D Camera Positions - RenderMe360 s3_all<br>' +
             '<sub>Rotate: Left click + drag | Zoom: Scroll | Pan: Right click + drag</sub>',
        x=0.5,
        xanchor='center'
    ),
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data',
        camera=dict(
            eye=dict(x=2, y=2, z=1.5)
        )
    ),
    width=900,
    height=700,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

# Show the plot
fig.show()

## 3. Camera Viewing Directions Visualization

In [ ]:
# Visualize camera viewing directions
fig_directions = go.Figure()

# Add cameras
fig_directions.add_trace(go.Scatter3d(
    x=camera_df['x'],
    y=camera_df['y'],
    z=camera_df['z'],
    mode='markers',
    marker=dict(
        size=6,
        color=camera_df['z'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Height (m)")
    ),
    text=camera_df['id'].astype(str),
    name='Cameras',
    hovertemplate='Camera %{text}<extra></extra>'
))

# Add viewing direction arrows (optical axes)
for _, cam in camera_df.iterrows():
    # Scale the optical axis for visualization
    scale = 0.3
    end_x = cam['x'] + scale * cam['optical_x']
    end_y = cam['y'] + scale * cam['optical_y']
    end_z = cam['z'] + scale * cam['optical_z']
    
    fig_directions.add_trace(go.Scatter3d(
        x=[cam['x'], end_x],
        y=[cam['y'], end_y],
        z=[cam['z'], end_z],
        mode='lines',
        line=dict(color='blue', width=2),
        opacity=0.6,
        showlegend=False,
        hoverinfo='skip'
    ))

# Add subject
fig_directions.add_trace(go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    marker=dict(size=10, color='red', symbol='diamond'),
    name='Subject'
))

fig_directions.update_layout(
    title='Camera Viewing Directions (Blue arrows show optical axes)',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data'
    ),
    width=900,
    height=700
)

fig_directions.show()

## 4. Top-Down View (Interactive 2D)

In [ ]:
# Create interactive 2D top-down view
fig_topdown = go.Figure()

# Add cameras
fig_topdown.add_trace(go.Scatter(
    x=camera_df['x'],
    y=camera_df['y'],
    mode='markers+text',
    marker=dict(
        size=12,
        color=camera_df['z'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Height (m)"),
        line=dict(width=1, color='black')
    ),
    text=camera_df['id'].astype(str),
    textposition="top center",
    name='Cameras',
    hovertemplate='<b>Camera %{text}</b><br>' +
                  'X: %{x:.2f}m<br>' +
                  'Y: %{y:.2f}m<br>' +
                  'Height: ' + camera_df['z'].round(2).astype(str) + 'm<br>' +
                  '<extra></extra>'
))

# Add subject at origin
fig_topdown.add_trace(go.Scatter(
    x=[0],
    y=[0],
    mode='markers',
    marker=dict(size=20, color='red', symbol='star'),
    name='Subject'
))

# Add circle to show average radius
theta_circle = np.linspace(0, 2*np.pi, 100)
avg_radius = camera_df['r_xy'].mean()
fig_topdown.add_trace(go.Scatter(
    x=avg_radius * np.cos(theta_circle),
    y=avg_radius * np.sin(theta_circle),
    mode='lines',
    line=dict(color='gray', dash='dash'),
    opacity=0.5,
    name='Avg radius',
    hoverinfo='skip'
))

fig_topdown.update_layout(
    title='Top-Down View of Camera Positions',
    xaxis_title='X (m)',
    yaxis_title='Y (m)',
    width=800,
    height=800,
    xaxis=dict(scaleanchor='y', scaleratio=1),
    showlegend=True
)

fig_topdown.show()

## 5. Camera Distribution Analysis

In [ ]:
# Create subplots for distribution analysis
from plotly.subplots import make_subplots

fig_dist = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Azimuth Distribution', 'Height Distribution',
                   'Radial Distance Distribution', 'Height vs Azimuth'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'scatter'}]]
)

# Azimuth distribution
fig_dist.add_trace(
    go.Bar(x=camera_df['id'], y=camera_df['azimuth'],
           marker_color='lightblue',
           hovertemplate='Camera %{x}: %{y:.1f}°<extra></extra>'),
    row=1, col=1
)

# Height distribution
fig_dist.add_trace(
    go.Bar(x=camera_df['id'], y=camera_df['z'],
           marker_color='lightgreen',
           hovertemplate='Camera %{x}: %{y:.2f}m<extra></extra>'),
    row=1, col=2
)

# Radial distance distribution
fig_dist.add_trace(
    go.Bar(x=camera_df['id'], y=camera_df['r_xy'],
           marker_color='lightcoral',
           hovertemplate='Camera %{x}: %{y:.2f}m<extra></extra>'),
    row=2, col=1
)

# Height vs Azimuth scatter
fig_dist.add_trace(
    go.Scatter(x=camera_df['azimuth'], y=camera_df['z'],
               mode='markers+text',
               marker=dict(size=10, color=camera_df['id'], colorscale='Viridis'),
               text=camera_df['id'].astype(str),
               textposition='top center',
               textfont=dict(size=8),
               hovertemplate='Camera %{text}<br>Azimuth: %{x:.1f}°<br>Height: %{y:.2f}m<extra></extra>'),
    row=2, col=2
)

# Update axes labels
fig_dist.update_xaxes(title_text="Camera ID", row=1, col=1)
fig_dist.update_yaxes(title_text="Azimuth (degrees)", row=1, col=1)

fig_dist.update_xaxes(title_text="Camera ID", row=1, col=2)
fig_dist.update_yaxes(title_text="Height (m)", row=1, col=2)

fig_dist.update_xaxes(title_text="Camera ID", row=2, col=1)
fig_dist.update_yaxes(title_text="Radial Distance (m)", row=2, col=1)

fig_dist.update_xaxes(title_text="Azimuth (degrees)", row=2, col=2)
fig_dist.update_yaxes(title_text="Height (m)", row=2, col=2)

fig_dist.update_layout(height=800, showlegend=False,
                      title_text="Camera Distribution Analysis")

fig_dist.show()

## 6. Cylindrical Coordinates View

In [ ]:
# Convert to cylindrical coordinates and create a "unwrapped" cylinder view
fig_cyl = go.Figure()

# Sort by azimuth for better visualization
camera_df_sorted = camera_df.sort_values('azimuth')

# Create unwrapped cylinder view (azimuth vs height)
fig_cyl.add_trace(go.Scatter(
    x=camera_df_sorted['azimuth'],
    y=camera_df_sorted['z'],
    mode='markers+text',
    marker=dict(
        size=15,
        color=camera_df_sorted['r_xy'],
        colorscale='Plasma',
        showscale=True,
        colorbar=dict(title="Radial Dist (m)"),
        line=dict(width=1, color='black')
    ),
    text=camera_df_sorted['id'].astype(str),
    textposition="top center",
    hovertemplate='<b>Camera %{text}</b><br>' +
                  'Azimuth: %{x:.1f}°<br>' +
                  'Height: %{y:.2f}m<br>' +
                  'Radial: ' + camera_df_sorted['r_xy'].round(2).astype(str) + 'm<br>' +
                  '<extra></extra>'
))

# Add grid lines for height levels
unique_heights = camera_df['z'].round(1).unique()
for h in unique_heights:
    fig_cyl.add_hline(y=h, line_dash="dot", line_color="gray", opacity=0.3)

# Add vertical lines every 30 degrees
for angle in range(-180, 181, 30):
    fig_cyl.add_vline(x=angle, line_dash="dot", line_color="gray", opacity=0.3)

fig_cyl.update_layout(
    title='Unwrapped Cylindrical View (Azimuth vs Height)',
    xaxis_title='Azimuth Angle (degrees)',
    yaxis_title='Height (m)',
    width=1000,
    height=600,
    xaxis=dict(range=[-180, 180], dtick=30),
    showlegend=False
)

fig_cyl.show()

# Print summary statistics
print("\nCamera Position Statistics:")
print("="*50)
print(f"Total cameras: {len(camera_df)}")
print(f"Height range: {camera_df['z'].min():.2f}m to {camera_df['z'].max():.2f}m")
print(f"Radial distance range: {camera_df['r_xy'].min():.2f}m to {camera_df['r_xy'].max():.2f}m")
print(f"Average radial distance: {camera_df['r_xy'].mean():.2f}m")
print(f"Azimuth range: {camera_df['azimuth'].min():.1f}° to {camera_df['azimuth'].max():.1f}°")

# Check if cameras surround subject
azimuth_range = camera_df['azimuth'].max() - camera_df['azimuth'].min()
if azimuth_range > 300:
    print("\n✓ Cameras SURROUND the subject (>300° coverage)")
else:
    print(f"\n✗ Cameras clustered in {azimuth_range:.1f}° arc")

## 7. Camera Subset Selection Visualization

In [ ]:
def visualize_camera_subset(num_cameras=12):
    """Visualize a subset of cameras for optimal coverage."""
    
    # Select cameras evenly distributed by azimuth
    sorted_df = camera_df.sort_values('azimuth')
    indices = np.linspace(0, len(sorted_df)-1, num_cameras, dtype=int)
    selected_ids = sorted_df.iloc[indices]['id'].values
    
    # Create visualization
    fig = go.Figure()
    
    # Add all cameras (gray)
    fig.add_trace(go.Scatter3d(
        x=camera_df['x'],
        y=camera_df['y'],
        z=camera_df['z'],
        mode='markers',
        marker=dict(size=5, color='lightgray', opacity=0.3),
        name='All cameras',
        hoverinfo='skip'
    ))
    
    # Add selected cameras (colored)
    selected_df = camera_df[camera_df['id'].isin(selected_ids)]
    fig.add_trace(go.Scatter3d(
        x=selected_df['x'],
        y=selected_df['y'],
        z=selected_df['z'],
        mode='markers+text',
        marker=dict(
            size=10,
            color=selected_df['z'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Height (m)")
        ),
        text=selected_df['id'].astype(str),
        textposition="top center",
        name=f'{num_cameras} selected',
        hovertemplate='<b>Camera %{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<br>Z: %{z:.2f}<extra></extra>'
    ))
    
    # Add subject
    fig.add_trace(go.Scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers',
        marker=dict(size=12, color='red', symbol='diamond'),
        name='Subject'
    ))
    
    fig.update_layout(
        title=f'Camera Subset Selection: {num_cameras} cameras<br>Selected IDs: {list(selected_ids)}',
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            aspectmode='data'
        ),
        width=900,
        height=700
    )
    
    fig.show()
    
    # Calculate coverage metrics
    selected_azimuths = sorted(selected_df['azimuth'].values)
    gaps = []
    for i in range(len(selected_azimuths)):
        next_i = (i + 1) % len(selected_azimuths)
        gap = selected_azimuths[next_i] - selected_azimuths[i]
        if gap < 0:
            gap += 360
        gaps.append(gap)
    
    print(f"\nCoverage Metrics for {num_cameras} cameras:")
    print(f"Selected Camera IDs: {list(selected_ids)}")
    print(f"Average gap: {np.mean(gaps):.1f}°")
    print(f"Max gap: {max(gaps):.1f}°")
    print(f"Min gap: {min(gaps):.1f}°")
    print(f"Gap std: {np.std(gaps):.1f}°")

# Interactive widget for camera selection
import ipywidgets as widgets
from IPython.display import display

camera_selector = widgets.IntSlider(
    value=12,
    min=4,
    max=20,
    step=2,
    description='# Cameras:',
    continuous_update=False
)

widgets.interact(visualize_camera_subset, num_cameras=camera_selector);

## 8. Export Data for Further Analysis

In [ ]:
# Export camera data to CSV for external use
output_path = Path('/ssd4/zhuoyuan/renderme360_temp/download_all/visualizations/camera_analysis')
output_path.mkdir(parents=True, exist_ok=True)

# Save full camera data
camera_df.to_csv(output_path / 'camera_positions_full.csv', index=False)
print(f"Camera data exported to: {output_path / 'camera_positions_full.csv'}")

# Create a summary
summary = {
    'total_cameras': len(camera_df),
    'height_range': [float(camera_df['z'].min()), float(camera_df['z'].max())],
    'radial_range': [float(camera_df['r_xy'].min()), float(camera_df['r_xy'].max())],
    'azimuth_range': [float(camera_df['azimuth'].min()), float(camera_df['azimuth'].max())],
    'camera_ids': camera_df['id'].tolist()
}

import json
with open(output_path / 'camera_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print(f"Summary exported to: {output_path / 'camera_summary.json'}")

print("\n" + "="*50)
print("Interactive exploration complete!")
print("You can now rotate, zoom, and explore the 3D visualizations above.")
print("Use the widgets to test different camera subset configurations.")